In [ ]:
!pip install transformers[torch]
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 37.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from sklearn.metrics import classification_report
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import spacy
!python -m spacy download es_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 39.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Cargar el modelo de spaCy para tokenizar oraciones
nlp = spacy.load("es_core_news_sm")


In [ ]:
# Función para obtener los enunciados con las keywords
def extract_keyword_sentences(row):
    keywords = set(row['keyword'].split(', '))  # Convertir las keywords en un conjunto para eliminar duplicados
    text = row['text']
    doc = nlp(text)
    found_sentences = set()  # Conjunto para almacenar las oraciones encontradas
    for sentence in doc.sents:
        # Verificar si la oración contiene al menos una de las keywords
        if any(keyword.lower() in sentence.text.lower() for keyword in keywords):
            found_sentences.add(sentence.text.strip())  # Agregar la oración al conjunto de oraciones encontradas
    return '. '.join(found_sentences) if found_sentences else None  # Unir las oraciones encontradas en un solo texto

In [ ]:
# Cargar los datos de entrenamiento
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[0, 1, 2, 3], engine='python')
# Filtrar el DataFrame para quedarse solo con las filas donde 'label' es diferente de 0
data_filtered = data.loc[data['label'] != 0]


In [ ]:
 # Filtrar y actualizar el texto con los enunciados que contienen las keywords
data_filtered['text'] = data_filtered.apply(lambda row: extract_keyword_sentences(row), axis=1)
# Convertir las etiquetas a un rango de 0 a 2
data_filtered['label'] -= 1


<ipython-input-9-8b25d6b0a273>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['text'] = data_filtered.apply(lambda row: extract_keyword_sentences(row), axis=1)
<ipython-input-9-8b25d6b0a273>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['label'] -= 1


In [ ]:
# Dividir los datos en entrenamiento y validación
train_df, val_df = train_test_split(data_filtered, test_size=0.2, random_state=42)


In [ ]:
# Estructurar los datos en formato de diccionario
train_data = {
    'text': train_df['text'].tolist(),
    'label': train_df['label'].tolist()
}

val_data = {
    'text': val_df['text'].tolist(),
    'label': val_df['label'].tolist()
}

In [ ]:
# Cargar el tokenizador y el modelo
tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest', truncation=True, max_lenght=512)
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Función para realizar el preprocesamiento de los datos
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512, padding=True)


In [ ]:
# Convertir los datos en objetos Dataset
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

# Aplicar el preprocesamiento a los datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/535 [00:00<?, ? examples/s]

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

In [ ]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)


In [ ]:
# Iniciar el entrenamiento
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Entrenar el modelo
trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.644224
2,1.373100,1.118114
3,0.897200,0.923941
4,0.897200,0.790036


TrainOutput(global_step=268, training_loss=1.0332547657525362, metrics={'train_runtime': 231.3288, 'train_samples_per_second': 9.251, 'train_steps_per_second': 1.159, 'total_flos': 563062713937920.0, 'train_loss': 1.0332547657525362, 'epoch': 4.0})

In [ ]:
import shutil
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

# Define la ruta de la carpeta que deseas guardar en Google Drive
carpeta_colab = '/content/results'  # Cambia esto por la ruta de tu carpeta en Colab
carpeta_drive = '/content/drive/MyDrive/FINE_SENTIMENT_1'  # Cambia esto por la ruta donde deseas guardar la carpeta en Drive

# Copia la carpeta de Colab a Drive
shutil.copytree(carpeta_colab, carpeta_drive)

Mounted at /content/drive


'/content/drive/MyDrive/FINE_SENTIMENT_1'

### **PRUEBA DE MODELO REEENTRENADO CON DATOS DE VALIDACIÓN CODALAB**

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, pipeline

# Ruta del modelo entrenado en Google Drive
modelo_entrenado_path = '/content/drive/MyDrive/FINE_SENTIMENT_1/checkpoint-268'

# Cargar el tokenizador y el modelo
tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')
model = RobertaForSequenceClassification.from_pretrained(modelo_entrenado_path)

# Crear el clasificador de sentimiento
sentiment_task = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)


In [ ]:
# Cargar los datos de validación
datos_validacion = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', usecols=[0,1,2,3])

# Filtrar el DataFrame para quedarse solo con las filas donde 'label' es diferente de 0
datos_validacion = datos_validacion.loc[datos_validacion['label'] != 0]


In [ ]:
 # Filtrar y actualizar el texto con los enunciados que contienen las keywords
datos_validacion['text'] = datos_validacion.apply(lambda row: extract_keyword_sentences(row), axis=1)

In [ ]:
# Aplicar el clasificador de sentimiento a los textos del archivo
resultados = []
for texto in datos_validacion['text']:
    output = sentiment_task(texto)
    resultados.append(output)

# Imprimir los resultados
for i, resultado in enumerate(resultados):
    print(f"Texto {i+1}: {resultado}")

Texto 1: [{'label': 'neutral', 'score': 0.6765705347061157}]
Texto 2: [{'label': 'neutral', 'score': 0.9574296474456787}]
Texto 3: [{'label': 'neutral', 'score': 0.8018009066581726}]
Texto 4: [{'label': 'neutral', 'score': 0.6241645216941833}]
Texto 5: [{'label': 'neutral', 'score': 0.6836655735969543}]
Texto 6: [{'label': 'neutral', 'score': 0.6055567264556885}]
Texto 7: [{'label': 'neutral', 'score': 0.7119956612586975}]
Texto 8: [{'label': 'positive', 'score': 0.49443313479423523}]
Texto 9: [{'label': 'positive', 'score': 0.36021366715431213}]
Texto 10: [{'label': 'neutral', 'score': 0.49724435806274414}]
Texto 11: [{'label': 'neutral', 'score': 0.8394274115562439}]
Texto 12: [{'label': 'neutral', 'score': 0.8630181550979614}]
Texto 13: [{'label': 'neutral', 'score': 0.8131124377250671}]
Texto 14: [{'label': 'negative', 'score': 0.5863868594169617}]
Texto 15: [{'label': 'neutral', 'score': 0.898335337638855}]
Texto 16: [{'label': 'neutral', 'score': 0.625194251537323}]
Texto 17: [{'

In [ ]:
print(resultados)

[[{'label': 'neutral', 'score': 0.6765705347061157}], [{'label': 'neutral', 'score': 0.9574296474456787}], [{'label': 'neutral', 'score': 0.8018009066581726}], [{'label': 'neutral', 'score': 0.6241645216941833}], [{'label': 'neutral', 'score': 0.6836655735969543}], [{'label': 'neutral', 'score': 0.6055567264556885}], [{'label': 'neutral', 'score': 0.7119956612586975}], [{'label': 'positive', 'score': 0.49443313479423523}], [{'label': 'positive', 'score': 0.36021366715431213}], [{'label': 'neutral', 'score': 0.49724435806274414}], [{'label': 'neutral', 'score': 0.8394274115562439}], [{'label': 'neutral', 'score': 0.8630181550979614}], [{'label': 'neutral', 'score': 0.8131124377250671}], [{'label': 'negative', 'score': 0.5863868594169617}], [{'label': 'neutral', 'score': 0.898335337638855}], [{'label': 'neutral', 'score': 0.625194251537323}], [{'label': 'neutral', 'score': 0.5549052953720093}], [{'label': 'neutral', 'score': 0.7762933969497681}], [{'label': 'neutral', 'score': 0.82619357

In [ ]:
# Función para mapear las etiquetas
def map_label(label):
    if label == 'positive':
        return 1
    elif label == 'neutral':
        return 2
    elif label == 'negative':
        return 3
    else:
        return None

In [ ]:
# Mapear las etiquetas de los resultados
mapeados = []
for resultado in resultados:
    etiqueta_original = resultado[0]['label']
    etiqueta_mapeada = map_label(etiqueta_original)
    mapeados.append(etiqueta_mapeada)

In [ ]:
print(mapeados)

[2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 3, 3, 2, 2, 3, 2, 2, 3, 2, 3, 2, 2, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 3, 3, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2]


In [ ]:
from sklearn.metrics import classification_report

y_true = datos_validacion['label']
y_pred = mapeados

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.17      0.07      0.10        54
           2       0.67      0.95      0.78       131
           3       0.00      0.00      0.00        38

    accuracy                           0.57       223
   macro avg       0.28      0.34      0.29       223
weighted avg       0.43      0.57      0.48       223

